## Ipython Notebooks Templates

Add the package path for testing

In [1]:
import sys
sys.path.append('E:\\git_folder\\quant_models')

## Start Testing

In [2]:
from util.metrics import sharpe_iid, compute_deflated_sharpe_ratio, \
        sharpe_non_iid, sharpe_iid_adjusted          

#### Part 1 Simulate Return Matrix

The logics is given as:

1. Generate N random algos that each list consists of 1, 0, -1
2. Apply the N algos on the XU1 price to get the return 
3. Resample the PNL to daily PNL

In [3]:
import pandas as pd
from util.utils import sample_df, sample_pnl
import numpy as np
import random
random.seed(50)
np.random.seed(50)

In [4]:
import pandas as pd
data_fn = "././data/xu1_1min.csv"
df_price = pd.read_csv(data_fn)

In [5]:
test_df = df_price.copy()
test_df.loc[:,'datetime'] = pd.to_datetime(test_df.datetime)
test_df.set_index('datetime', inplace=True)
test_df = test_df.loc[test_df.index > pd.to_datetime("2017-01-01"),:]

In [6]:
df_5min = sample_df(test_df, "30T")

In [7]:
df_5min['pct'] = df_5min.open.pct_change()
df_5min.dropna(inplace=True)

In [8]:
N_data = df_5min.count()['open']

In [9]:
# random generate strategy
N_stat = 1000
bs_stats = [np.random.randint(3, size=N_data) -1 for idx in range(N_stat)]
bs_stats = np.array(bs_stats).transpose()

In [10]:
print(bs_stats.shape)

(31444, 1000)


In [11]:
df_5min = df_5min[['pct']]

In [12]:
df_5min.shape

(31444, 1)

In [13]:
for idx in range(N_stat):
    # trade signl * pct will be the pnl
    col_name = 'strat_{}'.format(idx)
    df_5min[col_name] = bs_stats[:,idx]
    df_5min[col_name] = df_5min['pct'] * df_5min[col_name]

### 

In [14]:
df_5min.drop(columns=['pct'], inplace=True)

In [15]:
df_5min.head()

,strat_0,strat_1,strat_2,strat_3,strat_4,strat_5,strat_6,strat_7,strat_8,strat_9,...,strat_990,strat_991,strat_992,strat_993,strat_994,strat_995,strat_996,strat_997,strat_998,strat_999
datetime,,,,,,,,,,,,,,,,,,,,,
2017-01-03 09:00:00,-0.000000,-0.000000,-0.000000,0.000000,-0.000000,0.000000,-0.000000,-0.000000,-0.000000,-0.000000,...,0.000000,-0.000000,0.00000,0.000000,0.000000,-0.000000,-0.000000,-0.000000,0.000000,0.000000
2017-01-03 09:30:00,-0.001754,0.001754,0.000000,-0.001754,0.001754,0.001754,-0.001754,0.000000,-0.001754,0.000000,...,-0.001754,0.001754,0.00000,0.000000,0.001754,-0.001754,0.000000,-0.001754,-0.001754,0.001754
2017-01-03 10:00:00,0.000000,0.000000,-0.012253,0.000000,-0.012253,0.012253,0.000000,-0.012253,-0.012253,0.012253,...,0.000000,0.012253,0.00000,0.012253,0.000000,0.012253,-0.012253,-0.012253,-0.012253,0.000000
2017-01-03 10:30:00,0.000000,-0.002470,-0.002470,-0.002470,0.002470,-0.002470,0.002470,-0.002470,-0.002470,0.002470,...,-0.002470,-0.002470,-0.00247,-0.002470,0.002470,0.002470,-0.002470,-0.002470,0.002470,0.002470
2017-01-03 11:00:00,-0.004189,0.004189,-0.000000,0.004189,-0.000000,-0.004189,-0.000000,-0.004189,-0.000000,0.004189,...,-0.000000,-0.000000,-0.00000,-0.000000,-0.000000,0.004189,-0.000000,-0.000000,-0.000000,-0.000000


In [16]:
pnl_daily = sample_pnl(df_5min,'1D', cum_mode=False)

In [17]:
pnl_daily.head()

,strat_0,strat_1,strat_2,strat_3,strat_4,strat_5,strat_6,strat_7,strat_8,strat_9,...,strat_990,strat_991,strat_992,strat_993,strat_994,strat_995,strat_996,strat_997,strat_998,strat_999
datetime,,,,,,,,,,,,,,,,,,,,,
2017-01-03,-0.000486,0.003714,-0.010504,-0.001757,0.001379,0.006361,0.005167,-0.018658,-0.006823,0.021391,...,-0.007695,0.012777,0.002730,5.326021e-03,0.010425,0.012208,-0.014230,-1.572981e-02,-0.016479,0.009165
2017-01-04,-0.002982,-0.003198,-0.001485,-0.003928,-0.006171,-0.002722,0.001713,0.000708,0.001478,0.002705,...,-0.006418,-0.004702,0.001736,3.940992e-03,0.002698,-0.001238,0.005931,-3.209748e-03,0.006679,-0.006158
2017-01-05,-0.000496,-0.001481,0.001714,0.000006,-0.008826,-0.001723,-0.001225,0.003180,0.000485,-0.007855,...,0.001235,0.000241,0.006148,-8.099976e-03,-0.001478,0.001960,-0.001965,-5.395710e-03,0.001712,-0.004424
2017-01-06,-0.004922,-0.000732,-0.002221,0.006906,0.003446,0.001240,0.001728,0.003431,0.003436,-0.002957,...,0.004172,-0.001221,0.008118,-3.201678e-03,0.000734,-0.004430,0.004181,-1.472083e-03,-0.002700,0.002940
2017-01-07,0.000494,0.000988,-0.001235,0.000000,0.000741,0.000741,-0.000741,-0.000494,-0.000247,0.001235,...,-0.000988,-0.000988,-0.001482,1.218724e-07,0.001729,-0.001482,-0.000741,1.219026e-07,-0.001235,0.000494


In [18]:
# Split Pnl_daily into insample and outsample
pnl_insample  = pnl_daily.loc[pnl_daily.index <= pd.to_datetime('2017-06-01'), :]
pnl_outsample = pnl_daily.loc[pnl_daily.index >  pd.to_datetime('2017-06-01'), :]

In [19]:
pnl_daily.head()

,strat_0,strat_1,strat_2,strat_3,strat_4,strat_5,strat_6,strat_7,strat_8,strat_9,...,strat_990,strat_991,strat_992,strat_993,strat_994,strat_995,strat_996,strat_997,strat_998,strat_999
datetime,,,,,,,,,,,,,,,,,,,,,
2017-01-03,-0.000486,0.003714,-0.010504,-0.001757,0.001379,0.006361,0.005167,-0.018658,-0.006823,0.021391,...,-0.007695,0.012777,0.002730,5.326021e-03,0.010425,0.012208,-0.014230,-1.572981e-02,-0.016479,0.009165
2017-01-04,-0.002982,-0.003198,-0.001485,-0.003928,-0.006171,-0.002722,0.001713,0.000708,0.001478,0.002705,...,-0.006418,-0.004702,0.001736,3.940992e-03,0.002698,-0.001238,0.005931,-3.209748e-03,0.006679,-0.006158
2017-01-05,-0.000496,-0.001481,0.001714,0.000006,-0.008826,-0.001723,-0.001225,0.003180,0.000485,-0.007855,...,0.001235,0.000241,0.006148,-8.099976e-03,-0.001478,0.001960,-0.001965,-5.395710e-03,0.001712,-0.004424
2017-01-06,-0.004922,-0.000732,-0.002221,0.006906,0.003446,0.001240,0.001728,0.003431,0.003436,-0.002957,...,0.004172,-0.001221,0.008118,-3.201678e-03,0.000734,-0.004430,0.004181,-1.472083e-03,-0.002700,0.002940
2017-01-07,0.000494,0.000988,-0.001235,0.000000,0.000741,0.000741,-0.000741,-0.000494,-0.000247,0.001235,...,-0.000988,-0.000988,-0.001482,1.218724e-07,0.001729,-0.001482,-0.000741,1.219026e-07,-0.001235,0.000494


#### Part 2 Testing Sharpe ratio and its variants

* IID Sharpe
* IID Sharpe Adjusted
* Sharpe_non_iid

In [20]:
rtns_pct = pnl_daily.strat_105.values
print(type(rtns_pct))
print(rtns_pct.shape)

<class 'numpy.ndarray'>
(1080,)


In [21]:
# normal sharpe
sharpe = sharpe_iid(rtns_pct, bench=0, factor=252, log=False)
print(sharpe)

0.24542544330318658


In [22]:
# IID adjusted sharpe
adj_sharpe = sharpe_iid_adjusted(rtns_pct, bench=0, factor=252, log=False)
print(adj_sharpe)

0.24530880549200637


In [23]:
# None IID adjusted sharpe
nonIIDsharpe = sharpe_non_iid(rtns_pct, bench=0, factor=252, p_critical=0.05, log=False)
# since the algo is random, the Ljung-Box test for autocorrelation can be passed
# the value is the same as the normal sharpe
print(nonIIDsharpe)

0.4422447094531206


#### Part 3 Compute the deflated ration for the shapre

* it will consider the insample and outsample performance
* 
* from in_sample: the number of trials, the std of computed sharpe   
  -> reduce the selection bias
* the return sequence for estimated SR: the number of days,  the skew and kurtosis of the return sequence, the sharpe  
  -> reduce the non-normal bias
* The computed deflated sharpe is in the range (0, 1). The less, the better

In [24]:

sharpe_list = pnl_insample.apply(lambda x: sharpe_iid(x, factor=252, log=False))

In [25]:
sharpe_list.index

Index(['strat_0', 'strat_1', 'strat_2', 'strat_3', 'strat_4', 'strat_5',
       'strat_6', 'strat_7', 'strat_8', 'strat_9',
       ...
       'strat_990', 'strat_991', 'strat_992', 'strat_993', 'strat_994',
       'strat_995', 'strat_996', 'strat_997', 'strat_998', 'strat_999'],
      dtype='object', length=1000)

###  The best algo in the insample period

In [26]:
max_sharpe = sharpe_list.max()
std_sharpe = sharpe_list.std()
bset_strat = sharpe_list.idxmax()
nb_trials  = N_stat

In [27]:
print('best random algo is {} with a sharpe {}'.format(bset_strat, max_sharpe))

best random algo is strat_186 with a sharpe 3.9622877630897646


####  The selected algo in the outsample period

In [28]:
test_pnl_oos = pnl_outsample[bset_strat]
test_rtns = test_pnl_oos.values
test_pnl_is = pnl_insample[bset_strat]
test_rtns_is = test_pnl_is.values
estimated_sharpe = sharpe_iid(test_rtns, factor=252, log=False)

#### Compute the deflation of In-sample SR Estimation

In [29]:
print('sharpe computed: {}'.format(max_sharpe))

sharpe computed: 3.9622877630897646


In [30]:
ratio_deflated = compute_deflated_sharpe_ratio(max_sharpe, std_sharpe, nb_trials, test_rtns_is)

In [31]:
print(ratio_deflated)

0.0727152803789416


#### Compute the deflation of OOS SR Estimation

In [32]:
print('sharpe computed: {}'.format(estimated_sharpe))

sharpe computed: 0.4956008512229527


In [33]:
ratio_deflated = compute_deflated_sharpe_ratio(estimated_sharpe, std_sharpe, nb_trials, test_rtns)

In [34]:
print(ratio_deflated)

0.0


###  Pass the test

Since the best strat is totally data mining producted (randomly selected), it should not be used even if the in-sample sharpe is almost 4. Then, the computed deflated ratio is 0, which can filter the strategy as a false discovery. The ratio can be understood as the probability of the tested algo may have a postive sharpe in the future. The higher, the better